# Predicting creditcard approvals

## Plan<br>
0. Import packages
1. Investigate data
2. Clean data
3. EDA
4. Build model do this with a pipeline

## Import packages

We are probably going to use np, pd, plt<br>
Classification models --> knn, logistic-regression<br>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

## Investigate data

In [9]:
df = pd.read_csv('cc_approvals.data', names=['dummy_name_{}'.format(i) for i in range(16)])
df.head()

,dummy_name_0,dummy_name_1,dummy_name_2,dummy_name_3,dummy_name_4,dummy_name_5,dummy_name_6,dummy_name_7,dummy_name_8,dummy_name_9,dummy_name_10,dummy_name_11,dummy_name_12,dummy_name_13,dummy_name_14,dummy_name_15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
